In [1]:
from netCDF4 import Dataset
import numpy as np
import math
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd
from pandas.core.frame import DataFrame
import glob

/Users/monicazhu/Documents/Python/UrbanOH/urbanoh_env/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
tropomi_filepath = '/Volumes/share-wrf3/TROPOMI/NO2/*.nc'
tropomi_files = glob.glob(tropomi_filepath)

In [4]:
file_path = tropomi_files[0]
f = Dataset(file_path, mode='r')
lon = f.groups['PRODUCT'].variables['longitude'][0][:]
lat = f.groups['PRODUCT'].variables['latitude'][0][:]
time = f.groups['PRODUCT'].variables['time_utc'][0][:]
units = f.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'].units
NO2_trop = f.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'][0][:]

In [5]:
f.groups['PRODUCT'].variables.keys()

dict_keys(['scanline', 'ground_pixel', 'time', 'corner', 'polynomial_exponents', 'intensity_offset_polynomial_exponents', 'layer', 'vertices', 'latitude', 'longitude', 'delta_time', 'time_utc', 'qa_value', 'nitrogendioxide_tropospheric_column', 'nitrogendioxide_tropospheric_column_precision', 'nitrogendioxide_tropospheric_column_precision_kernel', 'averaging_kernel', 'air_mass_factor_troposphere', 'air_mass_factor_total', 'tm5_tropopause_layer_index', 'tm5_constant_a', 'tm5_constant_b'])

In [6]:
sites = ['BoulderCO','GreenbeltMD', 'MaunaLoaHI', 'NewHavenCT', 'ChapelHillNC', 
         'CharlesCityVA', 'FairbanksAK', 'HuntsvilleAL', 'KenoshaWI', 'ManhattanNY', 'MountainViewCA', 'NewBrunswickNJ', 'QueensNY', 'WashingtonDC']



In [3]:
def get_TROPOMI(file_path,longitude,latitude):
    ##The longitude and latitude variables are longitude and latitude of the pandora sites. 
    ##For BoulderCO it's longitude = -105.26 and latitude = 39.99, 
    ##for GreenbeltMD it's longitude = -76.8396, latitude = 38.9926,
    ##for MaunaLoaHI it's longitude = -155.6027, latitude = 19.4795,
    ##for NewHavenCT it's longitude = -72.9029, latitude = 41.3014
    ##I don't want to be so troublesome too, but I found that the four places can be contained in two different .nc files. :(
    f = Dataset(file_path, mode='r')
    lon = f.groups['PRODUCT'].variables['longitude'][0][:]
    lat = f.groups['PRODUCT'].variables['latitude'][0][:]
    time = f.groups['PRODUCT'].variables['time_utc'][0][:]
    units = f.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'].units
    NO2_trop = f.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'][0][:]
    NO2_trop_err = f.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column_precision'][0][:]
    list_1 = []
    for i in range(0,len(NO2_trop)):
        for j in range(0,len(NO2_trop[0])):
            if lon[i][j] >= longitude-0.5 and lon[i][j] <= longitude + 0.5:
                if lat[i][j] >= latitude-0.5 and lat[i][j] <= latitude+0.5:
                    if math.isnan(NO2_trop[i][j]) == False:
                        list_1.append((i,j))
    if len(list_1) == 0:
        print('all the qualified coincident data is NaN')
        return [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
    else:
            
        dist = []

        for i in list_1:
            dist.append((lon[i[0]][i[1]] - longitude)**2 + (lat[i[0]][i[1]] - latitude)**2)
        for i in range(len(dist)):
            if dist[i] == min(dist):
                dist_min = list_1[i]

        lon_min = lon[dist_min[0]][dist_min[1]]
        lat_min = lat[dist_min[0]][dist_min[1]]
        NO2_col_min = NO2_trop[dist_min[0]][dist_min[1]]
        NO2_col_err_min = NO2_trop_err[dist_min[0]][dist_min[1]]
        time_min = time[dist_min[0]]
        return [lon_min, lat_min, NO2_col_min, NO2_col_err_min, units, time_min]

In [5]:
boulder = []
for file in tropomi_files:
    try:
        this_data = get_TROPOMI(file,-105.26,39.99 )
        boulder.append(this_data)
    except:
        print('Can not read file {}'.format(file))
    np.save('/Users/monicazhu/Box/ACES/boulder.npy',np.array(boulder))

all the qualified coincident data is NaN
all the qualified coincident data is NaN


<ipython-input-3-22c319f45a77>:20: UserWarning: Warning: converting a masked element to nan.
  if math.isnan(NO2_trop[i][j]) == False:


all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualifie

all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
Can not read fil

all the qualified coincident data is NaN
all the qualified coincident data is NaN
Can not read file /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20190616T215541_20190616T233711_08676_01_010301_20190622T233921.nc
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
Can not read file /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20190901T174402_20190901T192532_09766_01_010302_20190907T200741.nc
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident 

all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
Can not read file /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20191205T161244_20191205T175413_11113_01_010302_20191211T160750.nc
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
Can not read file /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20190330T174932_20190330T193102_07567_01_010300_20190405T195053.nc
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident 

all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
Can not read file /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20190806T173332_20190806T191502_09397_01_010302_20190812T200626.nc
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
al

all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
Can not read file /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20190901T160233_20190901T174402_09765_01_010302_20190907T180208.nc
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
al

all the qualified coincident data is NaN
all the qualified coincident data is NaN
Can not read file /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20190222T190135_20190222T204305_07057_01_010202_20190228T210155.nc
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
al

all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
Can not read file /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20190327T152316_20190327T170447_07523_01_010300_20190402T165936.nc
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
Ca

In [ ]:
BoulderCO = []
for file in tropomi_files:
    try:
        this_data = get_TROPOMI(file,-105.26,39.99)
        BoulderCO.append(this_data)
    except:
        print('Fail to read {}'.format(file))
        continue
    np.save('/Users/monicazhu/Box/ACES/boulder.npy',BoulderCO)

Fail to read /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20190928T205750_20190928T223920_10151_01_010302_20191004T235951.nc
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the

all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
Fail to read /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20190325T192407_20190325T210538_0

all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
Fail to read /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20190901T174402_20190901T192532_09766_01_010302_20190907T200741.nc
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
Fail to read /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20190803T100322_20190803T114452_09350_01_010302_20190809T114835.nc
all the qualified coincident data is NaN
all the qualified coincident data is NaN
Fail to read /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20190611T045421

all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
Fail to read /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20190413T215208_20190413T233338_07768_01_010300_20190419T233335.nc
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is NaN
Fail to read /Volumes/share-wrf3/TROPOMI/NO2/S5P_OFFL_L2__NO2____20190825T181624_20190825T195754_09667_01_010302_20190831T204651.nc
all the qualified coincident data is NaN
all the qualified coincident data is NaN
all the qualified coincident data is Na

In [ ]:
NewHavenCT = []
for file in tropomi_files:
    try:
        this_data = get_TROPOMI(file,-72.9029, 41.3014)
        NewHavenCT.append(this_data)
    except:
        print('Fail to read {}'.format(file))
        continue

In [ ]:
GreenbeltMD = []
for file in tropomi_files:
    try:
        this_data = get_TROPOMI(file,-76.8396, 38.9926)
        GreenbeltMD.append(this_data)
    except:
        print('Fail to read {}'.format(file))
        continue

In [4]:
BoulderCO = get_TROPOMI("/Users/monicazhu/Box/ACES/S5P_OFFL_L2__NO2____20191106T184032_20191106T202201_10703_01_010302_20191112T214725.nc",-105.26,39.99)

In [5]:
NewHavenCT = get_TROPOMI("/Users/monicazhu/Box/ACES/S5P_OFFL_L2__NO2____20191106T165902_20191106T184032_10702_01_010302_20191112T191844.nc",-72.9029, 41.3014)

In [6]:
GreenbeltMD = get_TROPOMI("/Users/monicazhu/Box/ACES/S5P_OFFL_L2__NO2____20191106T165902_20191106T184032_10702_01_010302_20191112T191844.nc",longitude = -76.8396, latitude = 38.9926)

In [17]:
MaunaLoaHI = []
##It seems that it's not contained in any of the four files.

In [7]:
BoulderCO

[-105.23081,
 39.973965,
 0.00014924498,
 'mol m-2',
 '2019-11-06T19:46:46.324000Z']

In [19]:
NewHavenCT

[-73.07848, 41.35316, 3.50898e-05, 'mol m-2', '2019-11-06T18:05:16.780000Z']

In [20]:
GreenbeltMD

[-77.01803, 39.07455, 4.6375877e-05, 'mol m-2', '2019-11-06T18:04:50.741000Z']